In [1]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
import time
from collections import Counter
from skimage.feature import hog
from skimage import data, exposure
# from skimage.feature import greycomatrix, greycoprops
import math
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import skew,kurtosis
# from mahotas.features import surf
from skimage import measure
from skimage import feature, color
# import mahotas
import mimetypes
import matplotlib.pyplot as plt
from skimage.filters import gabor
from skimage.color import rgb2gray
from PIL import Image

In [2]:
root_path = 'D:\\ImageClassification\\dataset'
current_dir = os.getcwd()
os.chdir(root_path)
print(current_dir)

d:\ImageClassification


In [3]:
os.listdir()

['MHN', 'NDM', 'R2E2']

In [4]:
from os import walk

label = []
path = []
names = []

for folder_name in os.listdir():
    current_dir = os.path.join(root_path,folder_name)
    for root,dirs,files in walk(current_dir):
      for file in files:
        file_mimetype = mimetypes.guess_type(file)[0]
        if file_mimetype == 'image/jpeg':
          path.append(os.path.join(root,file))
          label.append(folder_name)
          names.append(file)
        

In [5]:
len(path), len(label)

(1544, 1544)

In [6]:
df = pd.DataFrame(np.array([path,names,label]).T,columns=['path','filename','label'])
df

,path,filename,label
0,D:\ImageClassification\dataset\MHN\Phrao Distr...,Phrao District_12_June 2023_MHN_10_IMG_9634_ro...,MHN
1,D:\ImageClassification\dataset\MHN\Phrao Distr...,Phrao District_12_June 2023_MHN_10_IMG_9635_ro...,MHN
2,D:\ImageClassification\dataset\MHN\Phrao Distr...,Phrao District_12_June 2023_MHN_11_IMG_9636_ro...,MHN
3,D:\ImageClassification\dataset\MHN\Phrao Distr...,Phrao District_12_June 2023_MHN_11_IMG_9637_ro...,MHN
4,D:\ImageClassification\dataset\MHN\Phrao Distr...,Phrao District_12_June 2023_MHN_12_IMG_9638_ro...,MHN
...,...,...,...
1539,D:\ImageClassification\dataset\R2E2\Singha Par...,Singha Park_8_June 2023_R2E2_8-2_IMG_9241_rota...,R2E2
1540,D:\ImageClassification\dataset\R2E2\Singha Par...,Singha Park_8_June 2023_R2E2_9-1_IMG_9242_rota...,R2E2
1541,D:\ImageClassification\dataset\R2E2\Singha Par...,Singha Park_8_June 2023_R2E2_9-1_IMG_9243_rota...,R2E2
1542,D:\ImageClassification\dataset\R2E2\Singha Par...,Singha Park_8_June 2023_R2E2_9-2_IMG_9244_rota...,R2E2


In [7]:
df.dtypes

path        object
filename    object
label       object
dtype: object

In [8]:
# Color Feature extraction Hsv and Rgb
def find_static_data(img,hsv=False):
  #b,g,r
  #h,s,v
  # It calculates the mean and standard deviation along the specified axes (0 and 1) of the input image
  mean = image_resize.mean(axis=(0,1))
  sd = image_resize.std(axis=(0,1))
  if hsv:
    h,s,v = cv2.split(img)
    # The Hue, Saturation, and Value channels are flattened into 1D arrays. Then, skewness (skew) and kurtosis (kurtosis) are calculated for each channel.
    flatted_H = h.flatten()
    flatted_S = s.flatten()
    flatted_V = v.flatten()

    s_h = skew(flatted_H)
    s_s = skew(flatted_S)
    s_v = skew(flatted_V)

    k_h = kurtosis(flatted_H)
    k_s = kurtosis(flatted_S)
    k_v = kurtosis(flatted_V)

    payload = {
      'mean' : mean,
      'std' : sd,
      'skew' : [s_h,s_s,s_v],
      'kurtosis' : [k_h,k_s,k_v]
    }
    
  else:
    b,g,r = cv2.split(img)
    # The Blue, Green, and Red channels are flattened into 1D arrays, and skewness and kurtosis are calculated for each channel.
    flatted_B = b.flatten()
    flatted_G = g.flatten()
    flatted_R = r.flatten()

    s_b = skew(flatted_B)
    s_g = skew(flatted_G)
    s_r = skew(flatted_R)

    k_b = kurtosis(flatted_B)
    k_g = kurtosis(flatted_G)
    k_r = kurtosis(flatted_R)

    payload = {
      'mean' : mean,
      'std' : sd,
      'skew' : [s_b,s_g,s_r],
      'kurtosis' : [k_b,k_g,k_r]
    }

  return payload

In [9]:
# Function for calculating LBP
# Texture feature extraction LBP
def lbp_calculated_pixel(img, x, y):
   
    center = img[x][y]
   
    val_ar = []
      
    # top_left
    val_ar.append(get_pixel(img, center, x-1, y-1))
      
    # top
    val_ar.append(get_pixel(img, center, x-1, y))
      
    # top_right
    val_ar.append(get_pixel(img, center, x-1, y + 1))
      
    # right
    val_ar.append(get_pixel(img, center, x, y + 1))
      
    # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y + 1))
      
    # bottom
    val_ar.append(get_pixel(img, center, x + 1, y))
      
    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y-1))
      
    # left
    val_ar.append(get_pixel(img, center, x, y-1))
       
    # Now, we need to convert binary
    # values to decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
   
    val = 0
      
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
          
    return val

In [10]:
def get_pixel(img, center, x, y):
      
    new_value = 0
      
    try:
        # If local neighbourhood pixel 
        # value is greater than or equal
        # to center pixel values then 
        # set it to 1
        if img[x][y] >= center:
            new_value = 1
              
    except:
        # Exception is required when 
        # neighbourhood value of a center
        # pixel value is null i.e. values
        # present at boundaries.
        pass
      
    return new_value

In [11]:
# Texture feature extraction Haralick
def compute_haralick_features(image, distances, angles):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the GLCM matrix
    glcm = graycomatrix(gray_image, distances, angles, levels=256, symmetric=True, normed=True)

    # Calculate the Haralick texture features
    features = []
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']:
        feature = np.mean(graycoprops(glcm, prop))
        features.append(feature)
        
    payload = {
      'Contrast' : features[0],
      'Dissimilarity' : features[1],
      'Homogeneity' : features[2],
      'Energy' : features[3],
      'Correlation' : features[4]
    }
    
    return payload


In [12]:
def extract_contour_features(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    threshold_value = 128  # Adjust this value based on your image characteristics

    # Apply thresholding to create a binary image
    _, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize a list to store contour-based features
    contour_features_list = []
    
    # Loop through each contour
    for contour in contours:
        # Calculate area of the contour
        area = cv2.contourArea(contour)
        
        # Calculate perimeter of the contour
        perimeter = cv2.arcLength(contour, True)
        
        # Calculate compactness (perimeter^2 / area)
        compactness = (perimeter ** 2) / area if area != 0 else 0.0
        
        # Calculate bounding box dimensions
        x, y, w, h = cv2.boundingRect(contour)
        
        # Calculate aspect ratio of the bounding box
        aspect_ratio = float(w) / h
        
        # Calculate extent (area of contour / area of bounding box)
        extent = area / (w * h)
        
        # Create a dictionary to store the features
        features_dict = {
            "Area": area,
            "Perimeter": perimeter,
            "Compactness": compactness,
            "Aspect Ratio": aspect_ratio,
            "Extent": extent
        }
        
        # Add the dictionary to the list
        contour_features_list.append(features_dict)
    
    return contour_features_list


In [13]:
def calculate_eccentricity(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to obtain a binary image
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the moments of the contour
    moments = cv2.moments(largest_contour)
    
    # Calculate the eccentricity using moments
    major_axis = 2 * ((moments["mu20"] + moments["mu02"] + ((moments["mu20"] - moments["mu02"])**2 + 4 * (moments["mu11"]**2))**0.5) / moments["m00"])**0.5
    minor_axis = 2 * ((moments["mu20"] + moments["mu02"] - ((moments["mu20"] - moments["mu02"])**2 + 4 * (moments["mu11"]**2))**0.5) / moments["m00"])**0.5
    
    # Calculate the eccentricity as the ratio of the major axis to the minor axis
    eccentricity = major_axis / minor_axis
    
    return eccentricity


In [14]:
def calculate_aspect_ratio(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find contours in the image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate aspect ratio for each contour
    aspect_ratios = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        aspect_ratios.append(aspect_ratio)
    
    return aspect_ratios

In [15]:
def calculate_circularity(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the image
    _, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours in the image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate circularity for each contour
    circularities = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            circularity = 0  # Set circularity to 0 if the perimeter is zero to avoid division by zero
        else:
            circularity = 4 * 3.14159265358979323846 * area / (perimeter * perimeter)
        circularities.append(circularity)
    
    return circularities

In [16]:
def calculate_solidity(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the image
    _, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours in the image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate solidity for each contour
    solidities = []
    for contour in contours:
        area = cv2.contourArea(contour)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0:
            solidity = 0  # Set solidity to 0 if the convex hull area is zero to avoid division by zero
        else:
            solidity = area / hull_area
        solidities.append(solidity)
    
    return solidities

In [17]:
# function to find Solidity 
def find_solidity(count): 
	contourArea = cv2.contourArea(count) 
	convexHull = cv2.convexHull(count) 
	contour_hull_area = cv2.contourArea(convexHull) 
	solidity = float(contourArea)/contour_hull_area 
	return solidity 


In [18]:
def calculate_compactness(contour):
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    compactness = (perimeter ** 2) / area
    return compactness

def extract_compactness(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate compactness for each contour and store the values
    compactness_values = []
    for contour in contours:
        compactness = calculate_compactness(contour)
        compactness_values.append(compactness)

    return compactness_values

In [19]:
def analyze_image(image):
    # convert the image to grayscale image 
    grayScaleImg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

    # apply thresholding to convert 
    # grayscale image to the binary image 
    ret, thresh = cv2.threshold(grayScaleImg, 40, 255, 0) 

    # find the contours 
    contours, hierarchy = cv2.findContours(thresh, 
                                        cv2.RETR_TREE, 
                                        cv2.CHAIN_APPROX_SIMPLE) 
    print(f"{len(contours)} objects detected") 

    # select first contour 
    count = contours[0] 

    # find the solidity for this contour 
    Solidity = find_solidity(count) 
    Solidity = round(Solidity, 2) 

    # find the equivalent diameter for this contour 
    equi_diameter = find_equi_diameter(count) 
    equi_diameter = round(equi_diameter, 2) 
    features_dict = {
        "Solidity" : Solidity,
        'Equivalent_Diameter' : equi_diameter
    }

    print("Solidity", Solidity) 
    print("Equivalent Diameter - ", equi_diameter) 
    return features_dict


In [20]:
def calculate_compactness(contour):
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    compactness = (perimeter ** 2) / area
    return compactness

In [21]:
import cv2
import numpy as np

def calculate_feret_diameter(contour):
    if len(contour) < 5:
        return None  # Return None if the contour does not have enough points
    _, _, _, (w, h) = cv2.minAreaRect(contour)
    feret_diameter = max(w, h)
    return feret_diameter

def extract_feret_diameter(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate Feret diameter for each contour and store the values
    feret_diameter_values = []
    for contour in contours:
        feret_diameter = calculate_feret_diameter(contour)
        if feret_diameter is not None:
            feret_diameter_values.append(feret_diameter)

    return feret_diameter_values

In [22]:
# GLCM Display function

def contrast_feature(matrix_coocurrence):
	contrast = graycoprops(matrix_coocurrence, 'contrast')
	return contrast

def dissimilarity_feature(matrix_coocurrence):
	dissimilarity = graycoprops(matrix_coocurrence, 'dissimilarity')	
	return dissimilarity

def homogeneity_feature(matrix_coocurrence):
	homogeneity = graycoprops(matrix_coocurrence, 'homogeneity')
	return homogeneity

def energy_feature(matrix_coocurrence):
	energy = graycoprops(matrix_coocurrence, 'energy')
	return energy

def correlation_feature(matrix_coocurrence):
	correlation = graycoprops(matrix_coocurrence, 'correlation')
	return correlation

def asm_feature(matrix_coocurrence):
	asm = graycoprops(matrix_coocurrence, 'ASM')
	return asm

In [23]:
import cv2
import numpy as np
import pywt

def texture_features_wavelet(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Perform Wavelet Transform
    coeffs = pywt.dwt2(gray_image, 'haar')
    cA, (cH, cV, cD) = coeffs
    
    # Calculate statistical features for each coefficient
    features = []
    for coeff in [cA, cH, cV, cD]:
        features.extend([
            np.mean(coeff),             # Mean
            np.std(coeff),              # Standard Deviation
            np.median(coeff),           # Median
            np.min(coeff),              # Minimum
            np.max(coeff),              # Maximum
            np.sum(np.abs(coeff)),      # Sum of absolute values
            np.sum(coeff ** 2)          # Sum of squares
        ])
    
    return features

In [24]:
def fractal_dimension(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold the grayscale image
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    
    # Find non-zero pixels (foreground) in the binary image
    foreground_pixels = np.transpose(np.nonzero(binary_image))
    
    # Maximum box size
    max_box_size = min(binary_image.shape)
    
    # Initialize list to store counts of boxes containing foreground pixels
    box_counts = []
    
    # Perform box-counting method
    for box_size in range(1, max_box_size + 1):
        # Count boxes containing foreground pixels
        count = 0
        for row in range(0, binary_image.shape[0], box_size):
            for col in range(0, binary_image.shape[1], box_size):
                if np.any(binary_image[row:row + box_size, col:col + box_size]):
                    count += 1
        box_counts.append(count)
    
    # Linear regression to estimate fractal dimension
    x = np.log(range(1, max_box_size + 1))
    y = np.log(box_counts)
    coeffs = np.polyfit(x, y, 1)
    fractal_dim = -coeffs[0]  # Fractal dimension is the slope of the linear regression line
    
    return fractal_dim

In [25]:
def calculate_extent(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)  # Get the largest contour in the image

    # Calculate extent
    area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)
    bounding_box_area = w * h
    extent = area / bounding_box_area

    return extent

In [26]:
import cv2
import numpy as np

def calculate_convex_hull_area(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)  # Get the largest contour in the image

    # Find convex hull
    hull = cv2.convexHull(contour)

    # Calculate area of convex hull
    convex_hull_area = cv2.contourArea(hull)

    return convex_hull_area

In [27]:

for row,path in enumerate(df['path']):
  start = time.time()
  image = cv2.imread(path)
  # image_resize = cv2.resize(image,(500,500),interpolation = cv2.INTER_AREA)
  image_resize= image

  ec = calculate_eccentricity(image_resize)
  print(ec)
  df.loc[row,'eccentricity'] = ec

  # aspect_ratios = calculate_aspect_ratio(image_resize)
  # print(aspect_ratios[0])
  # df.loc[row,'aspect_ratios'] = aspect_ratios[0]

  # circularities = calculate_circularity(image_resize)
  # print("Circularities:", circularities)
  # # print(path)

  # # # Shape Feature Extraction Contour
  contour = extract_contour_features(image_resize)
  df.loc[row,'contourArea'] = contour[0]['Area']
  df.loc[row,'contourPerimeter'] = contour[0]['Perimeter']
  df.loc[row,'contourCompactness'] = contour[0]['Compactness']
  df.loc[row,'contourAspectRatio'] = contour[0]['Aspect Ratio']
  df.loc[row,'contourExtent'] = contour[0]['Extent']

  # solidities = analyze_image(image)
  # x = len(solidities)
  # print(x,"Solidities:", solidities)

  # df.loc[row,'Solidity'] = solidities['Solidity']
  # df.loc[row,'Equivalent_Diameter'] = solidities['Equivalent_Diameter']
  # compactness_values = extract_compactness(image)
  # print(compactness_values)
  # feret_diameter_values = extract_feret_diameter(image_resize)
  # print(feret_diameter_values)
  # # Print Feret diameter values
  # for i, feret_diameter in enumerate(feret_diameter_values):
  #     print(f"Contour {i+1} Feret Diameter: {feret_diameter}")

  # # # Texture Feature Extraction LBP
  height, width, _ = image_resize.shape
  img_gray = cv2.cvtColor(image_resize,cv2.COLOR_BGR2GRAY)
  img_lbp = np.zeros((height, width),np.uint8)
  for i in range(0, height):
    for j in range(0, width):
        img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
  vector_lbp = img_lbp.flatten()
  counted = Counter(vector_lbp)
  for key,value in counted.items():
    df.loc[row, f"lbp_{key}"] = value

  # # Texture Feature Extraction Haralick
  # distances = [1]  # Distance between pixels in the GLCM
  # angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles for GLCM computation
  # haralick_features = compute_haralick_features(image_resize, distances, angles)
  # print(haralick_features)
  # df.loc[row,'Contrast'] = haralick_features['Contrast']
  # df.loc[row,'Dissimilarity'] = haralick_features['Dissimilarity']
  # df.loc[row,'Homogeneity'] = haralick_features['Homogeneity']
  # df.loc[row,'Energy'] = haralick_features['Energy']
  # df.loc[row,'Correlation'] = haralick_features['Correlation']

  # Calculate extent
  # extent_value = calculate_extent(image_resize)

  # # Display extent value
  # print("Extent:", extent_value)
  # df.loc[row,'Extent'] = extent_value

  # convex_hull_area = calculate_convex_hull_area(image_resize)

  # # Display convex hull area
  # print("Convex Hull Area:", convex_hull_area)

  # #glcm
  # img_gray = cv2.cvtColor(image_resize,cv2.COLOR_BGR2GRAY)
  # bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
  # inds = np.digitize(img_gray, bins)
  # max_value = inds.max()+1
  # matrix_coocurrence = graycomatrix(inds, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=False, symmetric=False)
 
  # # # 0 degree
  # df.loc[row,'contrast_0'] = contrast_feature(matrix_coocurrence)[0][0]
  # df.loc[row,'dissimilarity_0'] = dissimilarity_feature(matrix_coocurrence)[0][0]
  # df.loc[row,'homogeneity_0'] = homogeneity_feature(matrix_coocurrence)[0][0]
  # df.loc[row,'energy_0'] = energy_feature(matrix_coocurrence)[0][0]
  # df.loc[row,'correlation_0'] = correlation_feature(matrix_coocurrence)[0][0]
  # df.loc[row,'asm_0'] = asm_feature(matrix_coocurrence)[0][0]

  # # # 90 degree
  # df.loc[row,'contrast_90'] = contrast_feature(matrix_coocurrence)[0][1]
  # df.loc[row,'dissimilarity_90'] = dissimilarity_feature(matrix_coocurrence)[0][1]
  # df.loc[row,'homogeneity_90'] = homogeneity_feature(matrix_coocurrence)[0][1]
  # df.loc[row,'energy_90'] = energy_feature(matrix_coocurrence)[0][1]
  # df.loc[row,'correlation_90'] = correlation_feature(matrix_coocurrence)[0][1]
  # df.loc[row,'asm_90'] = asm_feature(matrix_coocurrence)[0][1]

  # # # 180 degree
  # df.loc[row,'contrast_180'] = contrast_feature(matrix_coocurrence)[0][2]
  # df.loc[row,'dissimilarity_180'] = dissimilarity_feature(matrix_coocurrence)[0][2]
  # df.loc[row,'homogeneity_180'] = homogeneity_feature(matrix_coocurrence)[0][2]
  # df.loc[row,'energy_180'] = energy_feature(matrix_coocurrence)[0][2]
  # df.loc[row,'correlation_180'] = correlation_feature(matrix_coocurrence)[0][2]
  # df.loc[row,'asm_180'] = asm_feature(matrix_coocurrence)[0][2]

  # # # 135 degree
  # df.loc[row,'contrast_135'] = contrast_feature(matrix_coocurrence)[0][3]
  # df.loc[row,'dissimilarity_135'] = dissimilarity_feature(matrix_coocurrence)[0][3]
  # df.loc[row,'homogeneity_135'] = homogeneity_feature(matrix_coocurrence)[0][3]
  # df.loc[row,'energy_135'] = energy_feature(matrix_coocurrence)[0][3]
  # df.loc[row,'correlation_135'] = correlation_feature(matrix_coocurrence)[0][3]
  # df.loc[row,'asm_135'] = asm_feature(matrix_coocurrence)[0][3]


  # # wavelet
  # features = texture_features_wavelet(image_resize)
  
  # print(len(features),"Texture Features (Wavelet Transform):", features)
  # for index, value in enumerate(features):
  #   name='wavelet',index
  #   df.loc[row, f"lbp_{index}"] = value


  # fd = fractal_dimension(image_resize)
  # print("Fractal Dimension:", fd)
  # df.loc[row,'Fractal'] = fd



  # # # # Color Feature Extraction RGB
  # payload_rgb = find_static_data(image_resize)
 
  # df.loc[row,'R_mean'] = payload_rgb['mean'][2]
  # df.loc[row,'G_mean'] = payload_rgb['mean'][1]
  # df.loc[row,'B_mean'] = payload_rgb['mean'][0]
  # #Standard deviation
  # df.loc[row,'R_STD'] = payload_rgb['std'][2]
  # df.loc[row,'G_STD'] = payload_rgb['std'][1]
  # df.loc[row,'B_STD'] = payload_rgb['std'][0]
  # # Skewness
  # df.loc[row,'R_skewness'] = payload_rgb['skew'][2]
  # df.loc[row,'G_skewness'] = payload_rgb['skew'][1]
  # df.loc[row,'B_skewness'] = payload_rgb['skew'][0]
  # # Kurtosis
  # df.loc[row,'R_kurtosis'] = payload_rgb['kurtosis'][2]
  # df.loc[row,'G_kurtosis'] = payload_rgb['kurtosis'][1]
  # df.loc[row,'B_kurtosis'] = payload_rgb['kurtosis'][0]

  # Find static data of LAB color
#   lab =cv2.cvtColor(image_resize,cv2.COLOR_BGR2LAB)
#   payload_lab = find_static_data(lab,hsv=True)

# #    #Average
#   df.loc[row,'L_mean'] = payload_lab['mean'][0]
#   df.loc[row,'A_mean'] = payload_lab['mean'][1]
#   df.loc[row,'B_mean'] = payload_lab['mean'][2]
#   #Standard deviation
#   df.loc[row,'L_STD'] = payload_lab['std'][0]
#   df.loc[row,'A_STD'] = payload_lab['std'][1]
#   df.loc[row,'B_STD'] = payload_lab['std'][2]
#   # Skewness
#   df.loc[row,'L_skewness'] = payload_lab['skew'][0]
#   df.loc[row,'A_skewness'] = payload_lab['skew'][1]
#   df.loc[row,'B_skewness'] = payload_lab['skew'][2]
#   # Kurtosis
#   df.loc[row,'L_kurtosis'] = payload_lab['kurtosis'][0]
#   df.loc[row,'A_kurtosis'] = payload_lab['kurtosis'][1]
#   df.loc[row,'B_kurtosis'] = payload_lab['kurtosis'][2]

  # # # Color Feature Extraction HSV
  hsv = cv2.cvtColor(image_resize,cv2.COLOR_RGB2HSV)
  payload_hsv = find_static_data(hsv,hsv = True)

  #Average
  df.loc[row,'H_mean'] = payload_hsv['mean'][0]
  df.loc[row,'S_mean'] = payload_hsv['mean'][1]
  df.loc[row,'V_mean'] = payload_hsv['mean'][2]
  #Standard deviation
  df.loc[row,'H_STD'] = payload_hsv['std'][0]
  df.loc[row,'S_STD'] = payload_hsv['std'][1]
  df.loc[row,'V_STD'] = payload_hsv['std'][2]
  # Skewness
  df.loc[row,'H_skewness'] = payload_hsv['skew'][0]
  df.loc[row,'S_skewness'] = payload_hsv['skew'][1]
  df.loc[row,'V_skewness'] = payload_hsv['skew'][2]
  # Kurtosis
  df.loc[row,'H_kurtosis'] = payload_hsv['kurtosis'][0]
  df.loc[row,'S_kurtosis'] = payload_hsv['kurtosis'][1]
  df.loc[row,'V_kurtosis'] = payload_hsv['kurtosis'][2]
    
  print('finished task at '+ str(time.time() - start))



2.570220324480884


C:\Users\xzodu\AppData\Local\Temp\ipykernel_2788\4229326718.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[row, f"lbp_{key}"] = value
C:\Users\xzodu\AppData\Local\Temp\ipykernel_2788\4229326718.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[row, f"lbp_{key}"] = value
C:\Users\xzodu\AppData\Local\Temp\ipykernel_2788\4229326718.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

finished task at 0.7293691635131836
2.5456734635048632
finished task at 0.5336041450500488
2.4641597455008566
finished task at 0.5654375553131104
2.499508976422513
finished task at 0.534675121307373
2.3402570479652782
finished task at 0.5420417785644531
2.410885652565256
finished task at 0.5328657627105713
2.4997730108586365
finished task at 0.528834342956543
2.5167212949026814
finished task at 0.605004072189331
2.6198315981289233
finished task at 0.6004817485809326
2.5379101814791594
finished task at 0.4898242950439453
2.253569857599577
finished task at 0.5632655620574951
2.194022662733619
finished task at 0.5849673748016357
2.602318330653224
finished task at 0.479994535446167
2.5646158013106475
finished task at 0.5385735034942627
2.4326454505862696
finished task at 0.5767509937286377
2.5130078282716406
finished task at 0.5841879844665527
2.418659957133903
finished task at 0.5495305061340332
2.4125866136716816
finished task at 0.5415542125701904
2.499488682694665
finished task at 0.52

In [ ]:
df

In [ ]:
os.listdir()
print(os.getcwd())

In [ ]:
df = df.reindex(sorted(df.columns), axis=1)

In [ ]:
df

In [ ]:
df.to_csv('feature_extractions_data.csv',index=False)